In [10]:
import pandas as pd
from datetime import datetime, date

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Load all data at once

In [39]:
conditions = pd.read_csv("../data/csv/conditions.csv")
patients = pd.read_csv("../data/csv/patients.csv")
observations = pd.read_csv("../data/csv/observations.csv")
#care_plans = pd.read_csv("../data/csv/careplans.csv")
encounters = pd.read_csv("../data/csv/encounters.csv")
devices = pd.read_csv("../data/csv/devices.csv")
#supplies = pd.read_csv('../data/csv/supplies.csv')
procedures = pd.read_csv("../data/csv/procedures.csv")
medications = pd.read_csv("../data/csv/medications.csv")

In [40]:
# set your condition, this will be our prediction "target"
condition = 'Chronic congestive heart failure (disorder)'

# Preprocessing

In [41]:
# perform any processing that is needed for your features
patients = patients.rename(columns={'Id': 'PATIENT'})

# convert birthdate to datetime
patients['BIRTHDATE'] = pd.to_datetime(patients['BIRTHDATE'])

# calculate age
patients['AGE'] = patients['BIRTHDATE'].apply(lambda x : (datetime.now().year - x.year))

# Set your features 
### Two possible formats: (table name, column name) OR (table name, filter column, filter value, column name)

In [61]:
features = [('patients','AGE'), ('patients','GENDER'),
            ('observations', 'DESCRIPTION', 'Body Mass Index', 'VALUE'),
            ('observations', 'DESCRIPTION', 'Left ventricular Ejection fraction', 'VALUE'),
            ('observations', 'DESCRIPTION', 'Systolic Blood Pressure', 'VALUE'),
            ('observations', 'DESCRIPTION', 'Heart rate', 'VALUE')]

# Build your dataset based on your feature list

In [57]:
X = patients[['PATIENT']]

for feature in features:
    merge_col = 'PATIENT'    
    print(feature)
    
    if len(feature) == 2:
        X = X.merge(locals()[feature[0]][[merge_col, feature[1]]], on=merge_col)
        
    elif len(feature) == 4:
        table = locals()[feature[0]]
        tmp = table.loc[table[feature[1]]==feature[2]].copy()
        tmp[feature[3]] = pd.to_numeric(tmp[feature[3]])
        tmp = tmp.groupby('PATIENT')[feature[3]].mean().reset_index()
        tmp = tmp.rename(columns={feature[3]:feature[2]})  
        print(tmp[feature[2]].median())
        X = X.merge(tmp, on='PATIENT', how='left')
        X[feature[2]] = X[feature[2]].fillna(X[feature[2]].median())
X

('patients', 'AGE')
('patients', 'GENDER')
('observations', 'DESCRIPTION', 'Body Mass Index', 'VALUE')
27.8
('observations', 'DESCRIPTION', 'Left ventricular Ejection fraction', 'VALUE')
39.55
('observations', 'DESCRIPTION', 'Systolic Blood Pressure', 'VALUE')
120.0
('observations', 'DESCRIPTION', 'Heart rate', 'VALUE')
97.48083333333332


,PATIENT,AGE,GENDER,Body Mass Index,Left ventricular Ejection fraction,Systolic Blood Pressure,Heart rate
0,1ff7f10f-a204-4bb1-aa72-dd763fa99482,5,M,16.5,39.55,112.333333,102.400000
1,9bcf6ed5-d808-44af-98a0-7d78a29ede72,6,F,16.2,39.55,120.000000,115.800000
2,5163c501-353c-4a82-b863-a3f1df2d6cf1,18,F,23.2,39.55,113.333333,100.700000
3,cc3c806f-4a09-4a89-a990-4286450956be,26,M,27.8,39.55,123.000000,72.200000
4,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,3,F,27.8,39.55,126.176471,108.923529
...,...,...,...,...,...,...,...
124145,1ecfda69-7afc-4417-8a6f-c00be1be96dc,88,M,27.3,36.55,122.000000,92.000000
124146,8cf835a7-f161-4fe9-a559-350c97a3450e,78,M,28.0,39.55,181.500000,57.450000
124147,503d768f-481c-46e2-bcdb-a6116686351a,88,M,27.9,39.55,119.352941,124.682353
124148,2599e9d9-ca59-44ec-a28c-9eae219f162d,88,M,27.9,39.00,121.000000,91.000000


In [58]:
# add your label
X = X.merge(conditions.loc[conditions['DESCRIPTION']==condition][['PATIENT','START']], on='PATIENT', how='left')
X = X.rename(columns={'START': 'label'})
X['label'] = X['label'].notnull()
X

,PATIENT,AGE,GENDER,Body Mass Index,Left ventricular Ejection fraction,Systolic Blood Pressure,Heart rate,label
0,1ff7f10f-a204-4bb1-aa72-dd763fa99482,5,M,16.5,39.55,112.333333,102.400000,False
1,9bcf6ed5-d808-44af-98a0-7d78a29ede72,6,F,16.2,39.55,120.000000,115.800000,False
2,5163c501-353c-4a82-b863-a3f1df2d6cf1,18,F,23.2,39.55,113.333333,100.700000,False
3,cc3c806f-4a09-4a89-a990-4286450956be,26,M,27.8,39.55,123.000000,72.200000,False
4,bd1c4ffc-7f1d-4590-adbb-1d6533fb623e,3,F,27.8,39.55,126.176471,108.923529,False
...,...,...,...,...,...,...,...,...
124145,1ecfda69-7afc-4417-8a6f-c00be1be96dc,88,M,27.3,36.55,122.000000,92.000000,True
124146,8cf835a7-f161-4fe9-a559-350c97a3450e,78,M,28.0,39.55,181.500000,57.450000,False
124147,503d768f-481c-46e2-bcdb-a6116686351a,88,M,27.9,39.55,119.352941,124.682353,False
124148,2599e9d9-ca59-44ec-a28c-9eae219f162d,88,M,27.9,39.00,121.000000,91.000000,True


# Save your preprocessed data to a CSV file

In [59]:
X.to_csv('../data/model_input.csv', index=False)